In [3]:
import pennylane as qml
from pennylane import numpy as np

import matplotlib.pyplot as plt

In [16]:
ising_datasets = qml.data.load("qspin", sysname = "Ising", periodicity = "closed", lattice = "chain", layout = "1x4")

In [21]:
hamiltonians = ising_datasets[0].hamiltonians
ground_states = ising_datasets[0].ground_states
ground_energies = ising_datasets[0].ground_energies

In [57]:
idx = 22

h_values = ising_datasets[0].parameters['h']
J = ising_datasets[0].parameters['J']

h = h_values[idx]
print(h)

gs = ground_states[idx]
gs_energy = ground_energies[idx]
rho = np.outer(gs, np.conjugate(gs)) #density matrix of gs

hamiltonians[idx]

0.888888888888889


(
    -1.0 * (Z(0) @ Z(1))
  + -1.0 * (Z(0) @ Z(3))
  + -1.0 * (Z(1) @ Z(2))
  + -1.0 * (Z(2) @ Z(3))
  + 0.888888888888889 * X(0)
  + 0.888888888888889 * X(1)
  + 0.888888888888889 * X(2)
  + 0.888888888888889 * X(3)
)

In [58]:
def build_tfim_hamiltonian(n, edges, J, h):
    """Return the TFIM Hamiltonian H = J Σ XX + h Σ Z."""
    coeffs = []
    ops = []

    # XX terms
    for i, j in edges:
        coeffs.append(J)
        #ops.append(qml.PauliX(i) @ qml.PauliX(j))
        ops.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # Z field terms
    for i in range(n):
        coeffs.append(h)
        #ops.append(qml.PauliZ(i))
        ops.append(qml.PauliX(i))

    return qml.Hamiltonian(coeffs, ops)

def chain_edges(n, periodic = False):
    edges = [(i, i+1) for i in range(n-1)]
    if periodic and n > 2:
        edges.append((n-1, 0))
    return edges

n = 4
edges = chain_edges(n, periodic = True)
H = build_tfim_hamiltonian(n, edges, J = J, h = h)
H

(
    -1 * (Z(0) @ Z(1))
  + -1 * (Z(1) @ Z(2))
  + -1 * (Z(2) @ Z(3))
  + -1 * (Z(3) @ Z(0))
  + 0.888888888888889 * X(0)
  + 0.888888888888889 * X(1)
  + 0.888888888888889 * X(2)
  + 0.888888888888889 * X(3)
)

In [59]:
def ground_state(ham): #to do : something something sparse matrix better
    """Return (E0, |psi0>) for a PennyLane Hamiltonian."""

    mat = qml.matrix(ham)
    eigvals, eigvecs = np.linalg.eigh(mat)

    idx = np.argmin(eigvals)
    return eigvals[idx], eigvecs[:, idx]


In [60]:
ground_state(H) 

(-4.951445697277298,
 array([-0.60377599+0.j,  0.1615669 +0.j,  0.1615669 +0.j, -0.11601866+0.j,
         0.1615669 +0.j, -0.06417512+0.j, -0.11601866+0.j,  0.1615669 +0.j,
         0.1615669 +0.j, -0.11601866+0.j, -0.06417512+0.j,  0.1615669 +0.j,
        -0.11601866+0.j,  0.1615669 +0.j,  0.1615669 +0.j, -0.60377599+0.j]))

In [61]:
gs_energy, gs

((-4.951445697277299+0j),
 array([-0.60377599+0.j,  0.1615669 +0.j,  0.1615669 +0.j, -0.11601866+0.j,
         0.1615669 +0.j, -0.06417512+0.j, -0.11601866+0.j,  0.1615669 +0.j,
         0.1615669 +0.j, -0.11601866+0.j, -0.06417512+0.j,  0.1615669 +0.j,
        -0.11601866+0.j,  0.1615669 +0.j,  0.1615669 +0.j, -0.60377599+0.j]))